In [732]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.gridspec as gridspec
import warnings
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_selector
from sklearn.preprocessing import MinMaxScaler
from fancyimpute import KNN
from sklearn.impute import KNNImputer
from fancyimpute import IterativeImputer
from scipy.stats import chi2_contingency

In [700]:
data = pd.read_csv('../raw_data/dataset.csv',sep=';')
data = data.copy()
data.head()

,uuid,default,account_amount_added_12_24m,account_days_in_dc_12_24m,account_days_in_rem_12_24m,account_days_in_term_12_24m,account_incoming_debt_vs_paid_0_24m,account_status,account_worst_status_0_3m,account_worst_status_12_24m,...,status_3rd_last_archived_0_24m,status_max_archived_0_6_months,status_max_archived_0_12_months,status_max_archived_0_24_months,recovery_debt,sum_capital_paid_account_0_12m,sum_capital_paid_account_12_24m,sum_paid_inv_0_12m,time_hours,worst_status_active_inv
0,63f69b2c-8b1c-4740-b78d-52ed9a4515ac,0.0,0,0.0,0.0,0.0,0.0,1.0,1.0,NaN,...,1,1,1,1,0,0,0,178839,9.653333,1.0
1,0e961183-8c15-4470-9a5e-07a1bd207661,0.0,0,0.0,0.0,0.0,NaN,1.0,1.0,1.0,...,1,1,2,2,0,0,0,49014,13.181389,NaN
2,d8edaae6-4368-44e0-941e-8328f203e64e,0.0,0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,...,1,1,2,2,0,0,0,124839,11.561944,1.0
3,0095dfb6-a886-4e2a-b056-15ef45fdb0ef,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,1,1,1,0,0,0,324676,15.751111,1.0
4,c8f8b835-5647-4506-bf15-49105d8af30b,0.0,0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,...,0,1,1,1,0,0,0,7100,12.698611,NaN


In [701]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99976 entries, 0 to 99975
Data columns (total 43 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   uuid                                 99976 non-null  object 
 1   default                              89976 non-null  float64
 2   account_amount_added_12_24m          99976 non-null  int64  
 3   account_days_in_dc_12_24m            88140 non-null  float64
 4   account_days_in_rem_12_24m           88140 non-null  float64
 5   account_days_in_term_12_24m          88140 non-null  float64
 6   account_incoming_debt_vs_paid_0_24m  40661 non-null  float64
 7   account_status                       45603 non-null  float64
 8   account_worst_status_0_3m            45603 non-null  float64
 9   account_worst_status_12_24m          33215 non-null  float64
 10  account_worst_status_3_6m            42274 non-null  float64
 11  account_worst_status_6_12m  

## 1. Preprocessing workflow 

### 1.1. Duplicates

In [702]:
# Duplicates
duplicate_count = data.duplicated().sum()
duplicate_count

0

 ### 1.2. Missing values 

In [703]:
# Missing data percentage
round((data.isnull().sum()/len(data)).sort_values(ascending=False),2)

worst_status_active_inv                0.70
account_worst_status_12_24m            0.67
account_worst_status_6_12m             0.60
account_incoming_debt_vs_paid_0_24m    0.59
account_worst_status_3_6m              0.58
account_status                         0.54
account_worst_status_0_3m              0.54
avg_payment_span_0_3m                  0.49
avg_payment_span_0_12m                 0.24
num_active_div_by_paid_inv_0_12m       0.23
num_arch_written_off_12_24m            0.18
num_arch_written_off_0_12m             0.18
account_days_in_rem_12_24m             0.12
account_days_in_term_12_24m            0.12
account_days_in_dc_12_24m              0.12
default                                0.10
sum_paid_inv_0_12m                     0.00
sum_capital_paid_account_12_24m        0.00
sum_capital_paid_account_0_12m         0.00
recovery_debt                          0.00
status_max_archived_0_24_months        0.00
status_max_archived_0_6_months         0.00
status_3rd_last_archived_0_24m  

In [704]:
_ = round((data.isnull().sum()/len(data)).sort_values(ascending=False),2)
miss_val_features_df = pd.DataFrame(_,columns=['percentage']).reset_index(names=['features'])
miss_val_features_df[miss_val_features_df['percentage'] >= 0.01]

,features,percentage
0,worst_status_active_inv,0.70
1,account_worst_status_12_24m,0.67
2,account_worst_status_6_12m,0.60
3,account_incoming_debt_vs_paid_0_24m,0.59
4,account_worst_status_3_6m,0.58
5,account_status,0.54
6,account_worst_status_0_3m,0.54
7,avg_payment_span_0_3m,0.49
8,avg_payment_span_0_12m,0.24
9,num_active_div_by_paid_inv_0_12m,0.23


### 1.3. Detect the MCAR pattern

> #### Detecting a MCAR pattern we can use statistical tests.
> #### We can use the qui-square test to test the goodness of fit, homogenity and independence 
> #### let's the first feature "worst_status_active_inv"
> #### alpha = 0.05 - the critical value 
> #### H_0: there is no association the chosen feature and the other ones


In [705]:
# The column is false
# data["account_incoming_debt_vs_paid_0_24m"] = False
# Replace where Height_missing with True where Height is missing
# data.loc[data[data["account_incoming_debt_vs_paid_0_24m"].isnull()].index, "account_incoming_debt_vs_paid_0_24m"] == True
data.loc[data["account_incoming_debt_vs_paid_0_24m"].isnull(), "account_incoming_debt_vs_paid_0_24m"] = True
data.loc[data["account_incoming_debt_vs_paid_0_24m"] != True, "account_incoming_debt_vs_paid_0_24m"] = False

> #### Let's see if the missing values of "account_worst_status_12_24m" are correlated with "sum_paid_inv_0_12m", which does not have missing values 

In [706]:
_ = data[data["account_incoming_debt_vs_paid_0_24m"]==False].groupby("merchant_group")["account_incoming_debt_vs_paid_0_24m"].count().value_counts().keys()
false_list = np.array(_)
false_list

array([  305,  2456,  8717,  1274, 17304,   308,   165,  3194,  1601,
         432,   536,  4365])

In [707]:
true_list = data[data["account_incoming_debt_vs_paid_0_24m"]==True].groupby("merchant_group")["account_incoming_debt_vs_paid_0_24m"].count().value_counts().keys()
true_list
# if len(true_list) == 0:
#    true_list = np.array(len(true_list))
#    true_list = np.zeros(false_list.shape)
# else:
#    true_list = np.array(true_list)

Int64Index([632, 2652, 8011, 1760, 31475, 439, 199, 4162, 2117, 690, 522,
            6660],
           dtype='int64')

In [708]:
# primeiro passo
# tenho de fazer o seguinte para as listas falsas e verdadeiras
# mesmo que seja zero a count é necessário que apareça 
# true_list e false_list têm de ter a mesma shape, sempre. 
# segundo passo 
# obter o p-value para todas as categorias

In [709]:
table = []
table.append(true_list)
table.append(false_list)


In [710]:
from scipy.stats import chi2_contingency
chi2, p, dof, ex = chi2_contingency(table)

In [711]:
print("The p-value is esqual to {}".format(p))

The p-value is esqual to 0.0


> #### Conclusion: We reject the null hypothesys. So, we **CANNOT** say there is **no association** they're independent

> #### Lets do the same for all features 

In [712]:
# selecting only the missing values features 
missing_features_only = miss_val_features_df[miss_val_features_df['percentage'] >= 0.1]['features']
missing_features_only


0                 worst_status_active_inv
1             account_worst_status_12_24m
2              account_worst_status_6_12m
3     account_incoming_debt_vs_paid_0_24m
4               account_worst_status_3_6m
5                          account_status
6               account_worst_status_0_3m
7                   avg_payment_span_0_3m
8                  avg_payment_span_0_12m
9        num_active_div_by_paid_inv_0_12m
10            num_arch_written_off_12_24m
11             num_arch_written_off_0_12m
12             account_days_in_rem_12_24m
13            account_days_in_term_12_24m
14              account_days_in_dc_12_24m
15                                default
Name: features, dtype: object

In [713]:
# analysing these missing values features
# below one can see the "account...status" are discrete
for i in missing_features_only[0:6+1]:
    print(data[missing_features_only][i].unique())

[ 1. nan  2.  3.]
[nan  1.  2.  3.  4.]
[nan  1.  3.  2.  4.]
[False True]
[ 1. nan  2.  3.  4.]
[ 1. nan  2.  3.  4.]
[ 1. nan  2.  3.  4.]


In [714]:
# as we can see the next some of the other features are also discrete
for i in missing_features_only[7:]:
    print(len(data[missing_features_only][i].unique()))
    

2257
7940
862
4
3
283
65
128
3


In [715]:
# according to what we said in "Dataset_explanation.ipynb" 
# these features means "number of archieved invoices that were unpaid between M and N months ago"
# I am attempted to consider them as discrete feature given the values we have
# probably, we have an upper limit for this situation. Meaning, customers are not allowed to have more than 2 unpaid invoices for a period of 24 months
# or 1 unpaid invoice for a period of 12 months
for i in missing_features_only[10:11+1]:
    print(data[missing_features_only][i].unique())

[ 0. nan  2.  1.]
[ 0. nan  1.]


In [716]:
# lets go further on this 
data[['uuid','num_arch_written_off_0_12m', 'num_arch_written_off_12_24m']][(data['num_arch_written_off_12_24m']  > 0) | (data['num_arch_written_off_0_12m'] > 0)]

,uuid,num_arch_written_off_0_12m,num_arch_written_off_12_24m
4736,fe583a43-ee49-4268-8ce5-1b1c53c38a0a,1.0,0.0
11939,0acd2796-3f77-4ce7-97ff-e3443d528fc7,1.0,0.0
19795,9b254660-b084-40b3-8e53-a2d11081f945,0.0,2.0
25071,08c0505e-a202-4627-970c-0e400af3807a,1.0,0.0
30081,cfb183fc-a502-408d-815d-cb9a4da3e110,0.0,1.0
44171,911b27da-977a-48bf-9354-548d3a52789e,0.0,1.0
50262,531f5e8e-b59b-4944-8215-4ba474a51bd2,1.0,0.0
54864,15d51024-2f68-4394-aa29-a909ed11609b,1.0,0.0
59024,9176cb03-325b-483e-9ffa-927358b782fe,0.0,1.0
59630,261e4f5e-724e-4540-b24b-73dfe1616578,1.0,0.0


In [717]:
# our discrete variables are 
discrete_missing_values = ['account_worst_status_12_24m','account_worst_status_6_12m',
                           'account_worst_status_3_6m','account_worst_status_0_3m',
                           'account_status','num_arch_written_off_12_24m',
                           'num_arch_written_off_0_12m','default']
discrete_missing_values

['account_worst_status_12_24m',
 'account_worst_status_6_12m',
 'account_worst_status_3_6m',
 'account_worst_status_0_3m',
 'account_status',
 'num_arch_written_off_12_24m',
 'num_arch_written_off_0_12m',
 'default']

In [718]:
continuos_missing_values = list(set(missing_features_only).difference(discrete_missing_values))
continuos_missing_values

['avg_payment_span_0_12m',
 'account_days_in_dc_12_24m',
 'account_incoming_debt_vs_paid_0_24m',
 'worst_status_active_inv',
 'avg_payment_span_0_3m',
 'num_active_div_by_paid_inv_0_12m',
 'account_days_in_rem_12_24m',
 'account_days_in_term_12_24m']

### 1.3.1. Feature Binning

In [719]:
def quantile(numeric_feature):
    
    numeric_feature = [float(x) for x in numeric_feature] # convert all values to float
    
    q1 = pd.Series(numeric_feature).quantile(0.25)
    q2 = pd.Series(numeric_feature).quantile(0.50)
    q3 = pd.Series(numeric_feature).quantile(0.75)
    
    for i in range(len(numeric_feature)):
        if numeric_feature[i] <= q1:
            numeric_feature[i] = '1st_quant'
        elif (numeric_feature[i] > q1) and (numeric_feature[i] <= q2):
            numeric_feature[i] = '2nd_quant'
        elif (numeric_feature[i] > q2) and (numeric_feature[i] <= q3):
            numeric_feature[i] = '3rd_quant'  
        else: 
            numeric_feature[i] = '4th_quant'
    return np.array(numeric_feature)

In [720]:
# for i in continuos_missing_values:
#    data[i] = quantile(data[i])

In [721]:
data = { i: quantile(data[i]) if i in continuos_missing_values else data[i] for i in data }.to


In [727]:
# Creates DataFrame.
data = pd.DataFrame(data)
data.head()

,uuid,default,account_amount_added_12_24m,account_days_in_dc_12_24m,account_days_in_rem_12_24m,account_days_in_term_12_24m,account_incoming_debt_vs_paid_0_24m,account_status,account_worst_status_0_3m,account_worst_status_12_24m,...,status_3rd_last_archived_0_24m,status_max_archived_0_6_months,status_max_archived_0_12_months,status_max_archived_0_24_months,recovery_debt,sum_capital_paid_account_0_12m,sum_capital_paid_account_12_24m,sum_paid_inv_0_12m,time_hours,worst_status_active_inv
0,63f69b2c-8b1c-4740-b78d-52ed9a4515ac,0.0,0,1st_quant,1st_quant,1st_quant,1st_quant,1.0,1.0,NaN,...,1,1,1,1,0,0,0,178839,9.653333,1st_quant
1,0e961183-8c15-4470-9a5e-07a1bd207661,0.0,0,1st_quant,1st_quant,1st_quant,2nd_quant,1.0,1.0,1.0,...,1,1,2,2,0,0,0,49014,13.181389,4th_quant
2,d8edaae6-4368-44e0-941e-8328f203e64e,0.0,0,1st_quant,1st_quant,1st_quant,2nd_quant,NaN,NaN,NaN,...,1,1,2,2,0,0,0,124839,11.561944,1st_quant
3,0095dfb6-a886-4e2a-b056-15ef45fdb0ef,0.0,0,4th_quant,4th_quant,4th_quant,2nd_quant,NaN,NaN,NaN,...,1,1,1,1,0,0,0,324676,15.751111,1st_quant
4,c8f8b835-5647-4506-bf15-49105d8af30b,0.0,0,1st_quant,1st_quant,1st_quant,2nd_quant,NaN,NaN,NaN,...,0,1,1,1,0,0,0,7100,12.698611,4th_quant


In [734]:
confusion_matrix = pd.crosstab(data.columns)

## Function to find Cramers coeff for categorical variables
def cramers_corrected_stat(confusion_matrix):
    """ calculate Cramers V statitic for categorical-categorical association"""
    
    chi2 - chi2_contingency (confusion_matrix) [e]
    n - confusion_matrix.sum(). sum()
    phi2 = chi2/n
    r,k - confusion_matrix. shape
    phi2corr - max(e, phi2 - ((k-1)* (r-1))/(n-1))
    rcorr - r - ((r-1)**2)/(n-1)
    kcorr - k - ((k-1)**2)/(n-1)
    
    return np.sqrt(phi2corr / min( (kcorr-1), (rcorr-1)))

# source: https://towardsdatascience.com/statistical-test-for-mcar-in-python-9fb617a76eac
# source: https://stackoverflow.com/questions/20892799/using-pandas-calculate-cram%C3%A9rs-coefficient-matrix

TypeError: crosstab() missing 1 required positional argument: 'columns'